# The code provided below is designed to demonstrate the data generated by the program pipeline

To begin with, the method _extract_required_metrics(report_path)_ is created that pulls all the necessary metrics from the initial reports 
and creates a data frame and a convenient table based on them:

In [ ]:
import pandas as pd
import os

# Function to extract required metrics from a report file
def extract_required_metrics(report_path):
    try:
        # Reading the report file
        df = pd.read_csv(report_path, sep='\t', header=None, names=['Metric', 'Value'])
        
        # Dictionary to store extracted metrics
        metrics = {}
        
        # List of key metrics to extract
        required_metrics = [
            ('Total length', 'Total length'),
            ('GC', 'GC (%)'),
            ('Largest contig', 'Largest Contig'),
            ('N50', 'N50'),
            ('N90', 'N90'),
            ('L50', 'L50'),
            ('L90', 'L90'),
            ('# N\'s per 100 kbp', '# N\'s per 100 kbp')
        ]
        
        # Extracting and saving the values of required metrics
        for metric_pattern, metric_name in required_metrics:
            metric_value = df[df['Metric'].str.contains(metric_pattern, case=False, regex=False)]['Value'].values
            if metric_value.size > 0:
                metrics[metric_name] = metric_value[0]
            else:
                metrics[metric_name] = 'N/A'  # Marking as 'N/A' if the metric is missing
        
        return metrics
    except Exception as e:
        print(f"Error processing {report_path}: {e}")
        return None

# Path to the directory containing report files
reports_dir = './reports'

# List of report files
report_files = [file for file in os.listdir(reports_dir) if file.endswith('.tsv')]

# Extracting and saving data from all report files
all_metrics_data = {}
for report_file in report_files:
    # Forming the full path to the report file
    report_path = os.path.join(reports_dir, report_file)
    # Determining the trimming parameter name from the report file name
    trimming_param = report_file.replace('report_scaffolds_', '').replace('.tsv', '')
    # Extracting metrics
    metrics = extract_required_metrics(report_path)
    if metrics:
        all_metrics_data[trimming_param] = metrics

# Converting the data into a DataFrame
all_metrics_df = pd.DataFrame.from_dict(all_metrics_data, orient='index').reset_index()
all_metrics_df.rename(columns={'index': 'Trimming Parameters'}, inplace=True)

# List of trimming parameters in the desired order
sorting_order = [
    "NoTrimming",
    "QualityTrim_Q25",
    "AdapterTrim_Q25",
    "LengthFilter_75",
    "ComplexityFilter",
    "SlidingWindow_4nt_q25",
    "QualitySlidingHybrid_Q25_4nt_q25", 
    "QualityAdapterHybrid_Q25", 
    "LengthComplexityHybrid_75", 
    "SlidingComplexityHybrid_4nt_q25",
    "AdapterSlidingHybrid_Q25_4nt_q25", 
    "QualityLengthHybrid_Q25_75", 
    "QualityComplexityHybrid_Q25", 
    "AdapterLengthHybrid_Q25_75", 
    "AdapterComplexityHybrid_Q25", 
    "LengthSlidingHybrid_75_4nt_Q25", 
]

# Sorting the DataFrame according to the specified order
sorted_metrics_df = all_metrics_df.set_index('Trimming Parameters').reindex(sorting_order).reset_index()

sorted_metrics_df


## After that, we would like to visualize the same data, but in a more expressive way.

To do this, we first define the base values for the **"NoTrimming"** Trimming Parameters, assuming that this is the control group or standard condition to which all other values will be compared. 

Next, we'll deal with data normalization: we'll initialize an empty _rows_list_ to store the normalized rows. In a loop over each row of the original DataFrame, computes the normalized deviations for each metric compared to the base values of the **"NoTrimming"** Trimming Parameters. If the base value of a metric is 0, the normalized deviation is set to 0 to avoid division by zero. 

Next, a new DataFrame _norm_deviations_ is created from the _rows_list_ of dictionaries, where each dictionary is a row with normalized values.

### Data visualisation:

Using _seaborn.heatmap()_, we create a heatmap to visualize the normalized deviations of genome assembly metrics relative to the **"NoTrimming"** method.

In the heatmap, the rows correspond to the different trimming methods (Trimming Parameters) and the columns correspond to the genome assembly metrics.

The colors of the heatmap cells reflect the magnitude of the normalized deviations, centered at 0, where shades of red indicate positive deviation and shades of blue indicate negative deviation.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Prepare the data
# Make sure the data is numerical.
for column in sorted_metrics_df.columns[1:]:  # Skip the first column with the names of trimming methods
    sorted_metrics_df[column] = pd.to_numeric(sorted_metrics_df[column], errors='coerce')

# Find the base values for "NoTrimming"
base_values = sorted_metrics_df[sorted_metrics_df['Trimming Parameters'] == 'NoTrimming'].iloc[0, 1:]

# Initialization of the list for storing strings
rows_list = []

# Cycle to fill the list of rows
for index, row in sorted_metrics_df.iterrows():
    norm_row = {'Trimming Parameters': row['Trimming Parameters']}
    for metric in sorted_metrics_df.columns[1:]:
        if base_values[metric] != 0:
            norm_row[metric] = (row[metric] - base_values[metric]) / base_values[metric]
        else:
            norm_row[metric] = 0
    rows_list.append(norm_row)

# Creating a DataFrame from a list of strings
norm_deviations = pd.DataFrame(rows_list, columns=sorted_metrics_df.columns)

# Visualization of normalized deviations using a heat map
plt.figure(figsize=(12, 8))
sns.heatmap(norm_deviations.iloc[:, 1:], annot=True, cmap='coolwarm', center=0, yticklabels=norm_deviations['Trimming Parameters'])
plt.title('Normalized Deviation of Genome Assembly Metrics from No Trimming')
plt.xlabel('Assembly Metrics')
plt.ylabel('Trimming Methods')
plt.xticks(rotation=45)  # Rotate signatures for better readability
plt.show()


### Analyzing the Normalized Deviation of Genome Assembly Metrics from No Trimming:

First of all, we observe that Total length and GC (%) values remain almost unchanged after applying different trimming methods. This may indicate that different data trimming methods produce consistent results for these key parameters. This is a good sign indicating the reproducibility and reliability of the genome assembly process. This may also indicate that trimming methods do not significantly affect the overall genome profile, at least in terms of total length and GC content. That is, trimming methods can effectively remove low-quality or undesirable portions of the data without losing significant genomic regions. The consistency of these metrics may also reflect the high quality of the original sequencing data. If the sequencing data were of high quality with minimal low-quality regions, trimming may have minimal impact on the overall length and GC content of the assembly.

The second thing that catches the eye is the red color in the heat map in all data where the **Sliding Window** trimming method was used: **SlidingWindow_4nt_q25**, **QualitySlidingHybrid_Q25_4nt_q25**, **SlidingComplexityHybrid_4nt_q25**, **AdapterSlidingHybrid_Q25_4nt_q25**, **LengthSlidingHybrid_75_4nt_Q25**.

Any combination with this trimming method leads to a rather large decrease in the N50 and N90 metrics relative to the value when trimming is not used. This indicates that this trimming method, in any chosen combination, leads to a decrease in genome assembly continuity and accuracy. Most likely, this trimming method may be too aggressive and remove significant portions of the sequenced data. It is possible that this trimming method results in the removal or alteration of DNA regions that are important for proper genome assembly, which affects the final length of the contigs.

In addition, all of the above combinations increase the L50 and L90 values. This means that more contigs are now required to reach 50% or 90% of the total genome assembly length, which may be the result of increased assembly fragmentation. This may indicate a decrease in assembly continuity and is likely due to the removal of significant data and the probable removal of overlaps required to assemble contigs into larger assemblies. 

**What conclusions can be drawn from this?**

1. If we notice an increase in L50 and L90 along with a decrease in N50 and N90 metrics, it is a clear sign that the trimming method leads to a decrease in assembly continuity and quality.
2. In such a case, it is worth reconsidering the trimming parameters or choosing another method that will have less impact on important parts of the data and maintain or improve the continuity of the assembly. For example, the methods named **LengthFilter_75** (filtering by length of reads), **ComplexityFilter** (the percentage of the base that is different from its next base) and their hybrid **LengthComplexityHybrid_75** performed best in terms of reducing the number of N's per 100 kbp relative to the situation when no filters are used (25% improvement). This translates into improved assembly quality, higher continuity, and potentially improved uncertainty.  
3. It is also important to consider the compromise between reducing errors and improving assembly continuity. In some cases, improving one aspect may lead to degradation of the other. For example, all 14 of the 15 combinations (except **LengthSlidingHybrid_75_4nt_Q25**) resulted in a decrease in the frequency of indeterminate nucleotides in the genome assembly for every 100,000 base pairs, but it also resulted in a decrease in the N50 parameter and an even greater decrease in the N90 parameter (an increase in the L50 and L90 parameters, respectively). Only the trimming method called **LengthFilter_75** again performed better than the others and showed the lowest reduction in the N50 parameter.
4. It would be a rational next step to construct an additional Plot to demonstrate the hierarchy of trimming methods with respect to their ratio of the frequency of occurrence of uncertain nucleotides to the parameter N50. To do this, we will create 16 bar plots that display the ratio of # N's per 100 kbp to N50 for each data trimming method and sort them into an descending hierarchy of these values.

In [ ]:
# Calculate the relationship and add it as a new column to the DataFrame multiplying the result by 10,000
sorted_metrics_df['N_Ratio_x10k'] = (sorted_metrics_df['# N\'s per 100 kbp'] / sorted_metrics_df['N50']) * 10000

# Sort the DataFrame by the new column 'N_Ratio'
sorted_df = sorted_metrics_df.sort_values(by='N_Ratio_x10k', ascending=False)

# Visualisation on one common graph
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Trimming Parameters', y='N_Ratio_x10k', data=sorted_df)

plt.title('N_Ratio (# N\'s per 100 kbp / N50) for Different Trimming Methods (Values x10,000)')
plt.xlabel('Trimming Methods')
plt.ylabel('N_Ratio (x10,000)')

# Change the colour of the 'NoTrimming' signature to red
for label in barplot.get_xticklabels():
    if label.get_text() == 'NoTrimming':
        label.set_color('red')

plt.xticks(rotation=45, ha='right')  # Tilt signatures for better readability

plt.tight_layout()
plt.show()

### Analyzing 16 bar plots "N_Ratio (# N\'s per 100 kbp / N50) for Different Trimming Methods (Values x10,000)":

Above all, if we consider the compromise between reducing the continuity of the assembly (decreasing metric N50) and reducing the number of uncertain nucleotides (denoted as 'N') in the genome assembly for every 100,000 base pairs (indicating improved assembly quality, with fewer gaps and uncertain sites in the genomic sequence), we can observe that all trimming methods have performed better than the situation where no trimming method is used, except for all combinations of the **Sliding Window** method. From this point of view, there is no justification for using these trimming methods. On the other hand, the trimming methods **LengthFilter_75**, **ComplexityFilter** and their hybrid **LengthComplexityHybrid_75** show the best result in this metric. 

**What conclusions can be drawn from this?**

1. In such a situation, it would be useful to examine in more detail the two extremes: the worst trimming method (**Sliding Window**) and the best trimming method with different parameters (**Length Filter**) in terms of this compromise. Then compare them in detail with the situation where no trimming is used (**NoTrimming**).
2. We run the programming pipeline again with the same raw sequencing data, but with a variety of different parameters for the two trimming methods selected above. The following 16 parameters for **Sliding Window** & 6 parameters for **Length Filter** were selected for the new run:
   - SlidingWindow_4nt_q15
   - SlidingWindow_4nt_q20
   - SlidingWindow_4nt_q25
   - SlidingWindow_4nt_q30
   - SlidingWindow_7nt_q15
   - SlidingWindow_7nt_q20
   - SlidingWindow_7nt_q25
   - SlidingWindow_7nt_q30
   - SlidingWindow_10nt_q15
   - SlidingWindow_10nt_q20
   - SlidingWindow_10nt_q25
   - SlidingWindow_10nt_q30
   - SlidingWindow_20nt_q15
   - SlidingWindow_20nt_q20
   - SlidingWindow_20nt_q25
   - SlidingWindow_20nt_q30
   - LengthFilter_50
   - LengthFilter_75
   - LengthFilter_100
   - LengthFilter_150
   - LengthFilter_200
   - LengthFilter_250

In [ ]:
# New path to the directory with reports
second_reports_dir = './reports_2'  

# List of report files in a new directory
second_report_files = [file for file in os.listdir(second_reports_dir) if file.endswith('.tsv')]

# Extracting and saving data from all report files
second_all_metrics_data = {}
for report_file in second_report_files:
    # Forming the full path to the report file
    second_report_path = os.path.join(second_reports_dir, report_file)
    # Determining the trimming parameter name from the second report file name
    second_trimming_param = report_file.replace('output_scaffolds_', '').replace('.tsv', '')
    # Extracting metrics
    second_metrics = extract_required_metrics(second_report_path)
    if second_metrics:
        second_all_metrics_data[second_trimming_param] = second_metrics

# Converting the data into a DataFrame
second_all_metrics_df = pd.DataFrame.from_dict(second_all_metrics_data, orient='index').reset_index()
second_all_metrics_df.rename(columns={'index': 'Trimming Parameters'}, inplace=True)

# List of trimming parameters in the desired order
second_sorting_order = [
    "NoTrimming",
    "SlidingWindow_4nt_q15",
    "SlidingWindow_4nt_q20",
    "SlidingWindow_4nt_q25",
    "SlidingWindow_4nt_q30",
    "SlidingWindow_7nt_q15",
    "SlidingWindow_7nt_q20", 
    "SlidingWindow_7nt_q25", 
    "SlidingWindow_7nt_q30", 
    "SlidingWindow_10nt_q15",
    "SlidingWindow_10nt_q20", 
    "SlidingWindow_10nt_q25", 
    "SlidingWindow_10nt_q30", 
    "SlidingWindow_20nt_q15", 
    "SlidingWindow_20nt_q20", 
    "SlidingWindow_20nt_q25",
    "SlidingWindow_20nt_q30",
    "LengthFilter_50",
    "LengthFilter_75",
    "LengthFilter_100",
    "LengthFilter_150",
    "LengthFilter_200",
    "LengthFilter_250",
]

# Sorting the DataFrame according to the specified order
second_sorted_metrics_df = second_all_metrics_df.set_index('Trimming Parameters').reindex(second_sorting_order).reset_index()

second_sorted_metrics_df